In this section, we will apply the MLflow concepts we have learned so far.

## Employee Attrition Detection

**Objective**  

Predict if an employee is likely to quit and identify the factors responsible - to allow HR to intervene on time and remedy the situation to prevent attrition.  While some level of attrition in a company is inevitable, minimizing it and being prepared for the cases that cannot be helped will significantly help improve the operations of most businesses.

**Data**  

The data set presents an employee survey from IBM, indicating if there is attrition or not. The data set contains approximately 1500 entries. Given the limited size of the data set, the model should only be expected to provide modest improvement in indentification of attrition vs a random allocation of probability of attrition.

## Import libraries

MLflow supports wide variety of ML frameworks. For this example we will use the simple and popular `sklearn` library for training a model. Additionally, for clarity, we have abstracted the steps for ML classification into a `kainos_sklearn` package.

In [ ]:
import warnings

import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler,
)

from kainos_sklearn.classifier import *

warnings.filterwarnings(action="ignore")

## Load data

The hr attrition data is available in csv format, so we can easily load it into a pandas dataframe.

In [ ]:
data = load_csv("./data/hr_attrition.csv")
n_rows, n_cols = data.shape
print(f"Rows: {n_rows}, Columns: {n_cols}")

## Data Config

The outcome of machine learning algorithm also depends on the tuning we apply to the data, hence it's important to keep a track of those too.

In [ ]:
drop_cols = [
    "EmployeeNumber",
    "EmployeeCount",
    "Over18",
    "StandardHours",
    "Attrition",
]

label_col = "Attrition"

test_size = 0.2
category_threshold = 8
feature_encoder = OrdinalEncoder
feature_scaler = RobustScaler

## ML Config

ML parameters are another import set of values to be tracked. Each algorithm has it's own parameters and our goal of our experiments is to find the parameters that works best for achieving our objective.

In [ ]:
tracking_uri = "http://localhost:5000"
experiment_name = "attrition_rf"
sklearn_classifier = RandomForestClassifier
model_params = dict(n_estimators=100, criterion="gini")

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment(experiment_name);

## Experiments

This is an iterative step. Here we infuse the code with MLflow to allow us to record the data/ml parameters and metrics. The idea is to set the values in previous two cells (data and ml config) and run the experiment. The process is to be repeated until desired results are achieved.

In [ ]:
with mlflow.start_run() as run:
    mlflow.log_param("feature_encoder", feature_encoder.__name__)
    mlflow.log_param("feature_scaler", feature_scaler.__name__)
    mlflow.log_param("sklearn_classifier", sklearn_classifier.__name__)
    mlflow.log_param("test_size", test_size)
    mlflow.log_params(model_params)
    train_X, train_y, test_X, test_y = split_data(
        data, drop_cols=drop_cols, label_col=label_col, test_size=test_size
    )
    model, labels_encoder, pipeline = train(
        train_X,
        train_y,
        feature_encoder,
        feature_scaler,
        sklearn_classifier,
        category_threshold,
    )
    mlflow.sklearn.log_model(model, "hr-attrition-model")
    accuracy, precision, recall, f1, cm, plt_cm = evaluate(
        model, labels_encoder, test_X, test_y
    )
    tp = cm[0][0]
    tn = cm[1][1]
    fp = cm[0][1]
    fn = cm[1][0]
    mlflow.log_metric("TP", tp)
    mlflow.log_metric("TN", tn)
    mlflow.log_metric("FP", fp)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("accuracy_score", accuracy)
    mlflow.log_metric("precision_score", precision)
    mlflow.log_metric("recall_score", recall)
    mlflow.log_metric("f1_score", f1)
    fig_name = "confusion-matrix.png"
    plt_cm.savefig(fig_name)
    mlflow.log_artifact(fig_name, "confusion-matrix-plot")